## Gendered Pronoun Resolution
### Pair pronouns to their correct entities
#### Qianqian QIAO and Xingce Bao

This notebook is for kaggle test Gendered Pronoun Resolution.  Very grateful for the kindly carrying from Qianqian!!!!

First we apply the bert to the test data

In [15]:
%load_ext watermark

In [2]:
import tensorflow as tf
print (tf.__version__)

1.12.0


Attention!!!

The minimum tensorflow version should be 1.10.

Run the bert_server with shell before continue.

I think the bert server has some bug since if you accidentally start the client before the server, the client will wait, and after you start the server, it makes all your computer slow down.....

Just remember to start the server before the notebook!!!!!

In [1]:
from bert_serving.client import BertClient
bc = BertClient()

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
gap_train = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv",sep='\t')
gap_test = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv",sep='\t')
gap_valid = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv",sep='\t')

In [6]:
gap_train.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera


In [7]:
test_data = pd.read_csv("./test_and_submit/test_stage_1.tsv",sep='\t')

In [8]:
test_data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera


In [9]:
data_num = len(list(test_data.Text.values)) # 2000

In [10]:
batch_size = 20
text = list(test_data.Text.values)
text_all = []
tokens_all = []

In [11]:
for i in tqdm(range(data_num//batch_size),ncols = 100):
    encoded = bc.encode(text[batch_size*i:batch_size*(i+1)],show_tokens=True)
    encoded_text = list(encoded[0])
    tokens = encoded[1]
    text_all.extend(encoded_text)
    tokens_all.extend(tokens)

100%|███████████████████████████████████████████████████████████| 100/100 [2:04:10<00:00, 83.47s/it]


In [12]:
test_data["vector"] = text_all
test_data["tokens"] = tokens_all

In [13]:
test_data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL,vector,tokens
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...,"[[-0.30144718, -0.08635384, -0.20015159, -0.25...","[[CLS], z, ##oe, te, ##lf, ##ord, -, -, played..."
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie,"[[0.34895068, 0.6811254, 0.92398053, -0.417637...","[[CLS], he, grew, up, in, e, ##van, ##ston, ,,..."
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...,"[[-0.3016174, 0.06391026, -0.15569207, -0.1670...","[[CLS], he, had, been, reelected, to, congress..."
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band),"[[0.31311065, -0.4944703, 0.44825977, -0.12736...","[[CLS], the, current, members, of, crime, have..."
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera,"[[0.1210996, 0.19799706, 0.4914001, -0.4640768...","[[CLS], her, sa, ##nta, f, ##e, opera, debut, ..."


In [19]:
test_data.to_pickle('./temp_result/test_kaggle')

In [25]:
data_num = len(list(gap_test.Text.values))

In [26]:
batch_size = 20
text = list(gap_test.Text.values)
text_all = []
tokens_all = []

In [27]:
for i in tqdm(range(data_num//batch_size),ncols = 100):
    encoded = bc.encode(text[batch_size*i:batch_size*(i+1)],show_tokens=True)
    encoded_text = list(encoded[0])
    tokens = encoded[1]
    text_all.extend(encoded_text)
    tokens_all.extend(tokens)

100%|███████████████████████████████████████████████████████████| 100/100 [2:02:33<00:00, 73.55s/it]


In [28]:
gap_test["vector"] = text_all
gap_test["tokens"] = tokens_all

In [29]:
gap_test.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,vector,tokens
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner,"[[0.33625355, -0.39955634, 1.5647364, -0.23100...","[[CLS], upon, their, acceptance, into, the, k,..."
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso,"[[0.38678372, -0.35543638, -0.4073126, -0.1891...","[[CLS], between, the, years, 1979, -, 1981, ,,..."
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh,"[[0.74173695, -0.60368216, 1.5596626, -0.00526...","[[CLS], though, his, emigration, from, the, co..."
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta,"[[0.6270518, 0.28286973, 0.55196077, -0.185908...","[[CLS], at, the, trial, ,, p, ##is, ##cio, ##t..."
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers,"[[0.40295172, -0.37557468, -0.8904216, -0.0598...","[[CLS], it, is, about, a, pair, of, united, st..."


In [30]:
gap_test.to_pickle('./temp_result/train_kaggle')

In [8]:
data_num = len(list(gap_valid.Text.values))

In [9]:
batch_size = 2
text = list(gap_valid.Text.values)
text_all = []
tokens_all = []

In [10]:
for i in tqdm(range(data_num//batch_size),ncols = 100):
    encoded = bc.encode(text[batch_size*i:batch_size*(i+1)],show_tokens=True)
    encoded_text = list(encoded[0])
    tokens = encoded[1]
    text_all.extend(encoded_text)
    tokens_all.extend(tokens)

100%|█████████████████████████████████████████████████████████████| 227/227 [25:31<00:00,  7.37s/it]


In [11]:
gap_valid["vector"] = text_all
gap_valid["tokens"] = tokens_all

In [12]:
gap_valid.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,vector,tokens
0,validation-1,He admitted making four trips to China and pla...,him,256,Jose de Venecia Jr,208,False,Abalos,241,False,http://en.wikipedia.org/wiki/Commission_on_Ele...,"[[0.04797277, -0.7341686, 0.5924454, -0.794429...","[[CLS], he, admitted, making, four, trips, to,..."
1,validation-2,"Kathleen Nott was born in Camberwell, London. ...",She,185,Ellen,110,False,Kathleen,150,True,http://en.wikipedia.org/wiki/Kathleen_Nott,"[[0.27766645, 0.38649005, -0.19211262, -0.2932...","[[CLS], ka, ##th, ##leen, not, ##t, was, born,..."
2,validation-3,"When she returns to her hotel room, a Liberian...",his,435,Jason Scott Lee,383,False,Danny,406,True,http://en.wikipedia.org/wiki/Hawaii_Five-0_(20...,"[[0.1443493, -0.8811491, 0.065385215, -0.37121...","[[CLS], when, she, returns, to, her, hotel, ro..."
3,validation-4,"On 19 March 2007, during a campaign appearance...",he,333,Reucassel,300,True,Debnam,325,False,http://en.wikipedia.org/wiki/Craig_Reucassel,"[[0.12763819, -0.15140754, -0.60868496, -0.862...","[[CLS], on, 19, march, 2007, ,, during, a, cam..."
4,validation-5,"By this time, Karen Blixen had separated from ...",she,427,Finch Hatton,290,False,Beryl Markham,328,True,http://en.wikipedia.org/wiki/Denys_Finch_Hatton,"[[0.3822317, -0.25520337, 0.16271535, 0.113445...","[[CLS], by, this, time, ,, ka, ##ren, b, ##lix..."


In [13]:
gap_valid.to_pickle('./temp_result/valid_kaggle')

In [16]:
%watermark 
%watermark --iversions
%watermark -a "by QianQian QIAO and Xingce BAO"

2019-02-20T20:19:48+01:00

CPython 3.6.8
IPython 7.2.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.10.0-40-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
numpy  1.14.6
pandas 0.24.1

by QianQian QIAO and Xingce BAO
